In [1]:
import numpy  as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

# for displaying variable w/o print
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# <a id='0'>Combining and Reshaping Data</a>
    
    
## Highlights

## Content
Wrangle data into a form that is more suitable for Data analysis and Machine Learning.

- <a href='#1'>Hierarchical Indexing</a>  
- <a href='#2'>Joining and Cancatenating</a>  
    - <a href='#21'> Database-style DataFrame Joins: Merge
    - <a href='#22'> Join: Merging on Index
    - <a href='#23'> Concatenating Along an Axis
- <a href='#3'>Reshaping Data: Pivoting</a>    
    - <a href='#32'> Pivoting "Long" to "Wide" Format
    - <a href='#33'> Pivoting "Wide" to "Long" Format

## <a id='1'> Hierarchical Indexing
Let's get warmed up by handling the multi-index of a **single DF** before we start to combine multiple DFs. <br>

### Create MultiIndex object

In [15]:
# from arrays
df1 = pd.DataFrame(np.random.randn(2), index=pd.MultiIndex.from_arrays([['Lab 1', 'Lab 2'], ['Test 1', 'Test 2']]) )
df1
# from product
df2 = pd.DataFrame(np.random.randn(4), index=pd.MultiIndex.from_product([['Lab 1', 'Lab 2'], ['Test 1', 'Test 2']]) )
df2

,,0
Lab 1,Test 1,0.593364
Lab 2,Test 2,-1.476764


0
Lab 1 Test 1  0.908384
      Test 2  1.375739
Lab 2 Test 1 -0.769270
      Test 2 -0.181230

In [18]:
df2.reindex(pd.MultiIndex.from_product([['Lab 1', 'Lab 2', 'Lab 3'], 
                                        ['Test 1', 'Test 2', 'Test 3']]), fill_value=np.nan)

0
Lab 1 Test 1  0.908384
      Test 2  1.375739
      Test 3       NaN
Lab 2 Test 1 -0.769270
      Test 2 -0.181230
      Test 3       NaN
Lab 3 Test 1       NaN
      Test 2       NaN
      Test 3       NaN

### Unstacking of Multi-Indexed Series

In [3]:
se = pd.Series(np.random.rand(9),
               index=[
                         ['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                         [1, 2, 3, 1, 3, 1, 2, 2, 3],
                         ['A', 'A', 'B', 'B', 'B', 'A', 'B', 'A', 'B'],
                       ]
              )
se.index.names = ['Patient','Test','Lab']
se

df = se.unstack() # se -> df, unstack the inner-most index.
df

Patient  Test  Lab
a        1     A      0.233085
         2     A      0.563843
         3     B      0.169504
b        1     B      0.773346
         3     B      0.105481
c        1     A      0.800954
         2     B      0.429252
d        2     A      0.417656
         3     B      0.499426
dtype: float64

Lab                  A         B
Patient Test                    
a       1     0.233085       NaN
        2     0.563843       NaN
        3          NaN  0.169504
b       1          NaN  0.773346
        3          NaN  0.105481
c       1     0.800954       NaN
        2          NaN  0.429252
d       2     0.417656       NaN
        3          NaN  0.499426

Access all information of patient a

In [6]:
df.loc['a']

Lab,A,B
Test,,
1,0.233085,NaN
2,0.563843,NaN
3,NaN,0.169504


### Challenge: combine two time series of 

### <a id='13'> Indexing with a DataFrame's column: column to index
ft. df.set_index( ) 
    
<a href='#0'> Back to TOC

In [8]:
# set_index, reset_index

frame = pd.DataFrame({'A': range(7), 
                      'B': range(7, 0, -1),
                      'C': ['one', 'one', 'one', 'two', 'two',
                            'two', 'two'],
                      'D': [0, 1, 2, 0, 1, 2, 3]})
frame

frame2 = frame.set_index(['C', 'D'])
frame2


,A,B,C,D
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


A  B
C   D      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

## <a id='2'> Joining and Cancatenating Datasets
 <a href='#0'> Go Back To TOC</a>

### <a id='21'> Database-Style Join of DataFrames

In [3]:
df1 = pd.DataFrame({'Key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'Data1': range(7)})
df2 = pd.DataFrame({'Key': ['a', 'b', 'd'],
                    'Data2': range(3)})

df1
df2

,Key,Data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


,Key,Data2
0,a,0
1,b,1
2,d,2


#### Merge two (and only two) DFs

In [5]:
# Make parameters explict: 
pd.merge(df1, df2, on='Key', how='right')

,Key,Data1,Data2
0,b,0.0,1
1,b,1.0,1
2,b,6.0,1
3,a,2.0,0
4,a,4.0,0
5,a,5.0,0
6,d,NaN,2


#### When the shared Key column has a different name in each df

In [6]:
df2 = df2.rename(columns={'Key': 'Key1'})

pd.merge(df1, df2.rename(columns={'Key1':'Key'}), how='left')

,Key,Data1,Data2
0,b,0,1.0
1,b,1,1.0
2,a,2,0.0
3,c,3,NaN
4,a,4,0.0
5,a,5,0.0
6,b,6,1.0


### <a id='22'> Merging on Index: ft. join
Join is a special merge when the shared key is the index for all the involved DFs. You can join DFs or named Series. You can also join multiple DFs all at once, unlike merge can only do two DFs at a time.
- <a href='#0'> Back to TOC

#### Join DFs

In [8]:
df1  = pd.DataFrame({'Key': ['c', 'b', 'a', 'a', 'b', 'c'], 'Value': range(6)})
df2  = pd.DataFrame({'Group_val': [3.5, 7]}, index=['b', 'a'])
df1
df2

,Key,Value
0,c,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


,Group_val
b,3.5
a,7.0


In [9]:
df1.set_index('Key').join(df2, how='outer') # set_index sorts the new index

,Value,Group_val
a,2,7.0
a,3,7.0
b,1,3.5
b,4,3.5
c,0,NaN
c,5,NaN


#### Join named Series

In [10]:
se1 = pd.Series(range(3), index=list('abc'))
se1

a    0
b    1
c    2
dtype: int64

In [11]:
# Note: no series.join
df1.set_index('Key').join(se1.rename('s1'), how='outer')

,Value,s1
a,2,0
a,3,0
b,1,1
b,4,1
c,0,2
c,5,2


#### Join multiple DFs

In [16]:
df3 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                       index=['a', 'c', 'e', 'f'],
                       columns=['New York', 'Oregon'])
df3

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [18]:
df1.set_index('Key').join([pd.DataFrame(se1.rename('s1')), df3], how='outer')

,Value,s1,New York,Oregon
a,2.0,0.0,7.0,8.0
a,3.0,0.0,7.0,8.0
b,1.0,1.0,NaN,NaN
b,4.0,1.0,NaN,NaN
c,0.0,2.0,9.0,10.0
c,5.0,2.0,9.0,10.0
e,NaN,NaN,11.0,12.0
f,NaN,NaN,16.0,17.0


### <a id='23'> Concatenating Along an Axis

Concatenate Series, DFs, or a mixture of both. <br>
It's widely used when engineering features during machine learning: you generate additional columns of features and horizontally stack them to the original DF. Or you have additional rows of data coming in, you need to stack them vertically. <br>
Note: it can be tricky when you horizontally stack multiple DFs that have different indices, as shown below.

<a href='#0'> Back to TOC

In [2]:
# on axis 0 by default, more rows
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3], index=['a', 'b'])
s3 = pd.Series([5, 6], index=['a', 'g'])
s1
s2
s3

a    0
b    1
dtype: int64

a    2
b    3
dtype: int64

a    5
g    6
dtype: int64

In [3]:
pd.concat([s1, s2, s3], axis=0) # vertical

a    0
b    1
a    2
b    3
a    5
g    6
dtype: int64

In [5]:
pd.concat([s1, s2, s3], axis=0, ignore_index=True) # ignore original index

0    0
1    1
2    2
3    3
4    5
5    6
dtype: int64

In [21]:
pd.concat([s1, s2, s3], axis=1) # horizontal, note the NAs created due to mismatching of the index.

,0,1,2
a,0.0,2.0,5.0
b,1.0,3.0,NaN
g,NaN,NaN,6.0


#### Ignore the original index of s3
If you simply want to stack the values and don't want matching index, then the NAs created above is not what you desire. The solution is to ignore the index of s2 and s3 by assigning s1's index to them.

In [34]:
s2.index = s1.index
s3.index = s1.index
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0,2,5
b,1,3,6


## <a id='3'>Pivoting
 <a href='#0'> Go Back To TOC</a>

### <a id='32'> Pivoting “Long” Format to “Wide” Format
"Long" data are commonly used in storing multiple time series into a database. The features are stacked into a single column instead of being unstacked into different columns. Pivot it into the unstacked ("wide") form so that we can apply further analysis.

In [8]:
long_df = pd.read_csv('examples/macrodata_cleaned_long.csv')
long_df.head()

,date,item,value
0,1959-03-31,realgdp,2710.349
1,1959-03-31,unemp,5.800
2,1959-03-31,infl,0.000
3,1959-06-30,realgdp,2778.801
4,1959-06-30,unemp,5.100


In [9]:
pivoted = long_df.pivot(index='date', columns='item', values='value')
pivoted.head()

item,infl,realgdp,unemp
date,,,
1959-03-31,0.00,2710.349,5.8
1959-06-30,2.34,2778.801,5.1
1959-09-30,2.74,2775.488,5.3
1959-12-31,0.27,2785.204,5.6
1960-03-31,2.31,2847.699,5.2


### <a id='33'> Melting “Wide” Format to “Long” Format
This time we reverse the process for data storage.
    
<a href='#0'> Back to TOC

In [10]:
pd.melt(pivoted.reset_index(), id_vars='date', value_vars=['infl', 'realgdp','unemp']).head()

,date,item,value
0,1959-03-31,infl,0.00
1,1959-06-30,infl,2.34
2,1959-09-30,infl,2.74
3,1959-12-31,infl,0.27
4,1960-03-31,infl,2.31
